In [1]:
import sys

PWD = '../../..'
sys.path.append(PWD)

import numpy as np
import pandas as pd

import tensorflow as tf

tf.get_logger().setLevel('ERROR')
from tensorflow import keras
from tensorflow.keras import optimizers
from tensorflow.keras import callbacks
import keras_tuner as kt

from src.network_utils import count_params
from src.cross_validator import KerasTunerCrossValidator
from src.dataset import load_dataset_train_test, load_dataset_train_val
from src.models import convnet_builder as bare_model_builder

In [2]:
PROJECT_NAME = 'convnet'

PLANE = 2
CHANNEL = 11

DATASET_PATH = PWD + '/data/dataset/dataset.pkl'
TRIALS_DIR = PWD + f'/data/model_selection/channel_{PLANE}_{CHANNEL}/tuner'
CROSSVAL_DIR = PWD + f'/data/model_selection/channel_{PLANE}_{CHANNEL}/cross_val'

LR = 0.01

N_EPOCHS = 3000
BATCH_SIZE = 2048
MAX_TRIALS = 40
EXECUTIONS_PER_TRIAL = 2

TOP_N = 5
CROSSVAL_N_CV = 5
CROSSVAL_N_EXEC = 2
LOSS_WEIGHT = 1000

In [3]:
X_base_train, _, y_base_train, _ = load_dataset_train_test(PWD, PLANE, CHANNEL)
X_train, X_val, y_train, y_val = load_dataset_train_val(PWD, PLANE, CHANNEL)

X_base_train.shape, X_train.shape, X_val.shape

((22412, 24), (17929, 24), (4483, 24))

# Model

In [4]:
def model_builder(hp: kt.HyperParameters) -> keras.Model:
    # Convolutional network params
    hp_n_conv_blocks = hp.Int("n_conv_blocks", min_value=1, max_value=3, step=1)
    hp_n_conv_layers = hp.Int("n_conv_layers", min_value=1, max_value=3, step=1)
    hp_filters_mult = hp.Choice("conv_filters_mult", values=[1, 2, 4, 8])
    hp_conv_spatial_dropout = hp.Choice("conv_spatial_dropout", values=[0.0, 0.1, 0.2])

    # MLP at the end params
    hp_mlp_n_hidden_layers = hp.Int("n_mlp_hidden_layers", min_value=0, max_value=3, step=1, default=0)
    hp_mlp_units_mult, hp_mlp_dropout = None, None
    if hp_mlp_n_hidden_layers > 0:
        hp_mlp_units_mult = hp.Choice("mlp_units_mult", values=[1, 2, 4, 8, 16], default=4)
        hp_mlp_dropout = hp.Choice("mlp_dropout", values=[0.0, 0.2, 0.5])

    # Other params
    hp_batch_normalization = hp.Boolean("batch_normalization", default=False)
    hp_input_batch_normalization = hp.Boolean("input_batch_normalization", default=False)

    model = bare_model_builder(hp_n_conv_blocks, hp_n_conv_layers, hp_filters_mult, hp_conv_spatial_dropout,
                               hp_mlp_n_hidden_layers, hp_mlp_units_mult, hp_mlp_dropout, hp_batch_normalization,
                               hp_input_batch_normalization)
    model.compile(loss='mse', optimizer=optimizers.Adam(LR), loss_weights=LOSS_WEIGHT)
    return model

In [5]:
model_builder(kt.HyperParameters()).summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape (Reshape)           (None, 24, 1)             0         
                                                                 
 conv1d (Conv1D)             (None, 24, 8)             32        
                                                                 
 max_pooling1d (MaxPooling1D  (None, 12, 8)            0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 96)                0         
                                                                 
 dense (Dense)               (None, 1)                 97        
                                                                 
Total params: 129
Trainable params: 129
Non-trainable params: 0
__________________________________________________________

In [6]:
model_callbacks = [
    callbacks.EarlyStopping(patience=50),
    callbacks.ReduceLROnPlateau(monitor='loss', factor=0.9, patience=10)
]

# Bayesian tuner

In [7]:
bayesian_tuner = kt.BayesianOptimization(model_builder, objective='val_loss', executions_per_trial=EXECUTIONS_PER_TRIAL,
                                         max_trials=MAX_TRIALS, directory=TRIALS_DIR,
                                         project_name=PROJECT_NAME, overwrite=False)

bayesian_tuner.search(X_train, y_train, validation_data=[X_val, y_val], epochs=N_EPOCHS, callbacks=model_callbacks,
                      batch_size=BATCH_SIZE, verbose=3)

# Results

In [8]:
bayesian_tuner.results_summary(1)

Results summary
Results in ../../../data/model_selection/channel_2_11/tuner\convnet
Showing 1 best trials
Trial summary
Hyperparameters:
n_conv_blocks: 1
n_conv_layers: 1
conv_filters_mult: 4
conv_spatial_dropout: 0.2
n_mlp_hidden_layers: 3
batch_normalization: False
input_batch_normalization: True
mlp_units_mult: 1
mlp_dropout: 0.0
Score: 8.680209159851074


In [9]:
best_model = bayesian_tuner.get_best_models(num_models=1)[0]
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization (BatchN  (None, 24)               96        
 ormalization)                                                   
                                                                 
 reshape (Reshape)           (None, 24, 1)             0         
                                                                 
 conv1d (Conv1D)             (None, 24, 32)            128       
                                                                 
 spatial_dropout1d (SpatialD  (None, 24, 32)           0         
 ropout1D)                                                       
                                                                 
 max_pooling1d (MaxPooling1D  (None, 12, 32)           0         
 )                                                               
                                                        

---

In [10]:
for i, hyperparameters in enumerate(bayesian_tuner.get_best_hyperparameters(TOP_N)):
    print(f'========== Model {i} ==========')
    print(hyperparameters.get_config()['values'])
    model_tmp = model_builder(hyperparameters)
    print('Number of parameters:', count_params(model_tmp))

========== Model 0 ==========
{'n_conv_blocks': 1, 'n_conv_layers': 1, 'conv_filters_mult': 4, 'conv_spatial_dropout': 0.2, 'n_mlp_hidden_layers': 3, 'batch_normalization': False, 'input_batch_normalization': True, 'mlp_units_mult': 1, 'mlp_dropout': 0.0}
Number of parameters: 6513
========== Model 1 ==========
{'n_conv_blocks': 1, 'n_conv_layers': 1, 'conv_filters_mult': 4, 'conv_spatial_dropout': 0.0, 'n_mlp_hidden_layers': 2, 'batch_normalization': True, 'input_batch_normalization': True, 'mlp_units_mult': 4, 'mlp_dropout': 0.2}
Number of parameters: 13201
========== Model 2 ==========
{'n_conv_blocks': 2, 'n_conv_layers': 2, 'conv_filters_mult': 2, 'conv_spatial_dropout': 0.0, 'n_mlp_hidden_layers': 1, 'batch_normalization': False, 'input_batch_normalization': True, 'mlp_units_mult': 4, 'mlp_dropout': 0.0}
Number of parameters: 8673
========== Model 3 ==========
{'n_conv_blocks': 1, 'n_conv_layers': 1, 'conv_filters_mult': 2, 'conv_spatial_dropout': 0.1, 'n_mlp_hidden_layers': 0, '

In [11]:
bayesian_tuner.results_summary(TOP_N)

Results summary
Results in ../../../data/model_selection/channel_2_11/tuner\convnet
Showing 5 best trials
Trial summary
Hyperparameters:
n_conv_blocks: 1
n_conv_layers: 1
conv_filters_mult: 4
conv_spatial_dropout: 0.2
n_mlp_hidden_layers: 3
batch_normalization: False
input_batch_normalization: True
mlp_units_mult: 1
mlp_dropout: 0.0
Score: 8.680209159851074
Trial summary
Hyperparameters:
n_conv_blocks: 1
n_conv_layers: 1
conv_filters_mult: 4
conv_spatial_dropout: 0.0
n_mlp_hidden_layers: 2
batch_normalization: True
input_batch_normalization: True
mlp_units_mult: 4
mlp_dropout: 0.2
Score: 8.708035469055176
Trial summary
Hyperparameters:
n_conv_blocks: 2
n_conv_layers: 2
conv_filters_mult: 2
conv_spatial_dropout: 0.0
n_mlp_hidden_layers: 1
batch_normalization: False
input_batch_normalization: True
mlp_units_mult: 4
mlp_dropout: 0.0
Score: 8.718840599060059
Trial summary
Hyperparameters:
n_conv_blocks: 1
n_conv_layers: 1
conv_filters_mult: 2
conv_spatial_dropout: 0.1
n_mlp_hidden_layers: 

# Cross-validation for top 5 models

In [12]:
cross_validator = KerasTunerCrossValidator(bayesian_tuner, X_base_train, y_base_train, model_builder,
                                           directory=CROSSVAL_DIR, project_name=PROJECT_NAME,
                                           n_epochs=N_EPOCHS, batch_size=BATCH_SIZE, n_top=TOP_N,
                                           n_cv=CROSSVAL_N_CV, n_executions=CROSSVAL_N_EXEC, overwrite=False)
model_scores = cross_validator()

{'n_conv_blocks': 1, 'n_conv_layers': 1, 'conv_filters_mult': 4, 'conv_spatial_dropout': 0.2, 'n_mlp_hidden_layers': 3, 'batch_normalization': False, 'input_batch_normalization': True, 'mlp_units_mult': 1, 'mlp_dropout': 0.0}
Number of parameters: 6513
Got score: 8.7255 (8.7430, 8.7079)
Got score: 8.4435 (8.4389, 8.4481)
Got score: 8.2383 (8.2794, 8.1973)
Got score: 15.8212 (23.0435, 8.5989)
Got score: 8.8322 (8.8358, 8.8285)


{'n_conv_blocks': 1, 'n_conv_layers': 1, 'conv_filters_mult': 4, 'conv_spatial_dropout': 0.0, 'n_mlp_hidden_layers': 2, 'batch_normalization': True, 'input_batch_normalization': True, 'mlp_units_mult': 4, 'mlp_dropout': 0.2}
Number of parameters: 13201
Got score: 8.7581 (8.7877, 8.7285)
Got score: 8.3915 (8.4092, 8.3737)
Got score: 8.2573 (8.1760, 8.3386)
Got score: 8.5090 (8.3843, 8.6338)
Got score: 8.9264 (8.8143, 9.0385)


{'n_conv_blocks': 2, 'n_conv_layers': 2, 'conv_filters_mult': 2, 'conv_spatial_dropout': 0.0, 'n_mlp_hidden_layers': 1, 'batch_normalization': False, 'input_batch_normalization': True, 'mlp_units_mult': 4, 'mlp_dropout': 0.0}
Number of parameters: 8673
Got score: 8.8197 (8.7731, 8.8664)
Got score: 8.4616 (8.3880, 8.5351)
Got score: 8.2369 (8.3219, 8.1519)
Got score: 8.9055 (9.3161, 8.4949)
Got score: 8.9530 (9.0448, 8.8612)


{'n_conv_blocks': 1, 'n_conv_layers': 1, 'conv_filters_mult': 2, 'conv_spatial_dropout': 0.1, 'n_mlp_hidden_layers': 0, 'batch_normalization': True, 'input_batch_normalization': True, 'mlp_units_mult': 8, 'mlp_dropout': 0.5}
Number of parameters: 337
Got score: 9.0116 (8.9301, 9.0932)
Got score: 8.4990 (8.5419, 8.4561)
Got score: 8.3671 (8.4168, 8.3174)
Got score: 8.6957 (8.7751, 8.6162)
Got score: 8.8894 (8.8633, 8.9156)


{'n_conv_blocks': 1, 'n_conv_layers': 3, 'conv_filters_mult': 2, 'conv_spatial_dropout': 0.2, 'n_mlp_hidden_layers': 3, 'batch_normalization': True, 'input_batch_normalization': True, 'mlp_units_mult': 2, 'mlp_dropout': 0.0}
Number of parameters: 8737
Got score: 8.8886 (8.9472, 8.8300)
Got score: 8.5753 (8.5621, 8.5886)
Got score: 8.2738 (8.3331, 8.2146)
Got score: 8.6041 (8.6253, 8.5830)
Got score: 9.0208 (8.9462, 9.0953)


In [13]:
mean_scores = [f"{np.mean(scores):0.2f}" for scores in model_scores.values()]
std_scores = [f"{np.std(scores):0.2f}" for scores in model_scores.values()]
n_params = [count_params(model_builder(hyperparameters)) for hyperparameters in
            bayesian_tuner.get_best_hyperparameters(TOP_N)]

df = pd.DataFrame({'mean': mean_scores, 'std': std_scores, 'n_params': n_params}, index=model_scores.keys())
df.index.name = 'Model'
df

,mean,std,n_params
Model,,,
0,10.01,2.91,6513
1,8.57,0.24,13201
2,8.68,0.28,8673
3,8.69,0.24,337
4,8.67,0.26,8737
